# Land Classifier Pipeline

This notebook runs the pipeline in ordered stages:
1. Data preparation
2. Rule-based model run
3. Rule-based output preview
4. Baseline + v2 + v3 model runs (train, infer, preview)

Notes:
- You may be prompted to authenticate Earth Engine during the first GEE call.
- Training and inference can take a long time. Run cells sequentially.
- Dependencies install from `requirements-colab.txt`.
- GEE script used for t-1, t, t+1 NDVI/EVI/BSI inspection: https://noble-truck-479712-a1.projects.earthengine.app/view/years-khajura


In [ ]:
from pathlib import Path
import os
import subprocess
import sys

REPO_URL = "https://github.com/KU-GEOI-525-2023/fallow-land-detection-khajura-bilstm.git"
REPO_DIR = Path.cwd() / "fallow-land-detection-khajura-bilstm"

if not REPO_DIR.exists():
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print(f"Repo already exists at {REPO_DIR}")

os.chdir(REPO_DIR)
print(f"Working directory: {Path.cwd()}")

subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements-colab.txt"], check=True)
src_path = Path.cwd() / "src"
os.environ["PYTHONPATH"] = str(src_path)
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


## 1. Data preparation


In [ ]:
import os

os.environ.setdefault("GEE_PROJECT_ID", "noble-truck-479712-a1")
print("GEE_PROJECT_ID set to", os.environ["GEE_PROJECT_ID"])


In [ ]:
from pathlib import Path
import subprocess
import sys

from omegaconf import OmegaConf
from IPython.display import Image, display

DATA_CONFIG = "configs/data/dataset_v2.yaml"
RULE_BASED_CONFIG = "configs/baselines/rule_based.yaml"
TRAIN_BASELINE_CONFIG = "configs/train/baseline.yaml"
TRAIN_V2_CONFIG = "configs/train/v2_optimized.yaml"
TRAIN_V3_CONFIG = "configs/train/v3_stable.yaml"
INFER_BASELINE_CONFIG = "configs/inference/baseline.yaml"
INFER_V2_CONFIG = "configs/inference/v2.yaml"
INFER_V3_CONFIG = "configs/inference/v3.yaml"

TRAIN_BASELINE_NAME = Path(TRAIN_BASELINE_CONFIG).stem
TRAIN_V2_NAME = Path(TRAIN_V2_CONFIG).stem
TRAIN_V3_NAME = Path(TRAIN_V3_CONFIG).stem

def run_cmd(args):
    print("+", " ".join(args))
    subprocess.run(args, check=True)

def run_rule_based():
    run_cmd([sys.executable, "-m", "land_classifier.baselines.rule_based_land_classifier", "--config", RULE_BASED_CONFIG])
    run_cmd([sys.executable, "-m", "land_classifier.evaluation.evaluate_rule_based", "--config", RULE_BASED_CONFIG])

def run_train(train_name):
    run_cmd([sys.executable, "-m", "land_classifier.training.train", f"train={train_name}"])

def run_infer(config_path):
    run_cmd([sys.executable, "-m", "land_classifier.inference.inference", "--config", config_path])

def run_plot(config_path):
    run_cmd([sys.executable, "-m", "land_classifier.visualization.plot_classification_map", "--config", config_path])

def show_classification_png(config_path: str) -> None:
    cfg = OmegaConf.load(config_path)
    plot_cfg = cfg.get("classification_plot")
    if not plot_cfg:
        print(f"Missing classification_plot in {config_path}")
        return
    output_path = Path(plot_cfg.get("output_path", ""))
    if output_path.exists():
        display(Image(filename=str(output_path)))
    else:
        print(f"Missing: {output_path}")


In [ ]:
run_cmd([sys.executable, "-m", "land_classifier.data.scripts.extract_training_data_v2", "--config", DATA_CONFIG])


## 2. Rule-based model run


In [ ]:
run_rule_based()


## 3. Rule-based output preview


In [ ]:
run_plot(RULE_BASED_CONFIG)


In [ ]:
show_classification_png(RULE_BASED_CONFIG)


## 4. Baseline + v2 + v3 model runs


In [ ]:
run_train(TRAIN_BASELINE_NAME)


In [ ]:
run_infer(INFER_BASELINE_CONFIG)


In [ ]:
run_plot(INFER_BASELINE_CONFIG)


In [ ]:
show_classification_png(INFER_BASELINE_CONFIG)


In [ ]:
run_train(TRAIN_V2_NAME)


In [ ]:
run_infer(INFER_V2_CONFIG)


In [ ]:
run_plot(INFER_V2_CONFIG)


In [ ]:
show_classification_png(INFER_V2_CONFIG)


In [ ]:
run_train(TRAIN_V3_NAME)


In [ ]:
run_infer(INFER_V3_CONFIG)


In [ ]:
run_plot(INFER_V3_CONFIG)


In [ ]:
show_classification_png(INFER_V3_CONFIG)
